In [33]:
import pandas as pd

In [34]:
df_BCD=pd.read_csv('../../../datasets/BreastCancerWisconsinDataSet.csv')
df_BCD[:2]

,id,diagnosis,radius_mean,texture_mean,perimeter_mean,area_mean,smoothness_mean,compactness_mean,concavity_mean,concave points_mean,...,texture_worst,perimeter_worst,area_worst,smoothness_worst,compactness_worst,concavity_worst,concave points_worst,symmetry_worst,fractal_dimension_worst,Unnamed: 32
0,842302,M,17.99,10.38,122.8,1001.0,0.11840,0.27760,0.3001,0.14710,...,17.33,184.6,2019.0,0.1622,0.6656,0.7119,0.2654,0.4601,0.11890,NaN
1,842517,M,20.57,17.77,132.9,1326.0,0.08474,0.07864,0.0869,0.07017,...,23.41,158.8,1956.0,0.1238,0.1866,0.2416,0.1860,0.2750,0.08902,NaN


### Quest
- 목표변수: diagnosis
- 설명변수: texture_worst: gray-scale 값의 std 대한 값
            smoothness_worst: radius lengths 
            concavity_worst: contour의 concave portions의 심각 정도의 값
- diagnosis 전처리(appy() M-0, B-1), 
- Logistic regression 학습, 평가, 서비스(pickle)

In [35]:
df_BCD.columns

Index(['id', 'diagnosis', 'radius_mean', 'texture_mean', 'perimeter_mean',
       'area_mean', 'smoothness_mean', 'compactness_mean', 'concavity_mean',
       'concave points_mean', 'symmetry_mean', 'fractal_dimension_mean',
       'radius_se', 'texture_se', 'perimeter_se', 'area_se', 'smoothness_se',
       'compactness_se', 'concavity_se', 'concave points_se', 'symmetry_se',
       'fractal_dimension_se', 'radius_worst', 'texture_worst',
       'perimeter_worst', 'area_worst', 'smoothness_worst',
       'compactness_worst', 'concavity_worst', 'concave points_worst',
       'symmetry_worst', 'fractal_dimension_worst', 'Unnamed: 32'],
      dtype='object')

### data 전처리

In [36]:
df_BCD_tsc_worst= df_BCD[['diagnosis', 'texture_worst','smoothness_worst','concavity_worst']]
df_BCD_tsc_worst.isnull().sum()

diagnosis           0
texture_worst       0
smoothness_worst    0
concavity_worst     0
dtype: int64

In [37]:
def convert(str) :
    if str == 'B' :
        return 1
    else :
        return 0

In [38]:
df_BCD_tsc_worst['diagnosis'] = df_BCD_tsc_worst['diagnosis'].apply(convert)

C:\Users\01-15\AppData\Local\Temp\ipykernel_10416\3125329743.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_BCD_tsc_worst['diagnosis'] = df_BCD_tsc_worst['diagnosis'].apply(convert)


In [39]:
df_BCD_tsc_worst.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 569 entries, 0 to 568
Data columns (total 4 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   diagnosis         569 non-null    int64  
 1   texture_worst     569 non-null    float64
 2   smoothness_worst  569 non-null    float64
 3   concavity_worst   569 non-null    float64
dtypes: float64(3), int64(1)
memory usage: 17.9 KB


In [40]:
target_worst = df_BCD_tsc_worst['diagnosis']
features_worst = df_BCD_tsc_worst[['texture_worst','smoothness_worst','concavity_worst']]
target_worst.shape, features_worst.shape

((569,), (569, 3))

### Learning Algorithm
- 목표변수: 범주형  
    ⇒ LogisticRegression 

In [41]:
from sklearn.linear_model import LogisticRegression

In [42]:
model = LogisticRegression()

In [43]:
model.fit(features_worst, target_worst)

LogisticRegression()

### 예측

In [44]:
df_BCD_tsc_worst[250:255]

,diagnosis,texture_worst,smoothness_worst,concavity_worst
250,0,27.00,0.1211,0.69910
251,1,22.46,0.1183,0.08105
252,0,25.59,0.1710,0.84890
253,0,25.09,0.1416,0.33780
254,0,24.57,0.1497,0.43170


In [45]:
model.predict(features_worst[250:255])

array([0, 1, 0, 1, 0], dtype=int64)

In [46]:
# 확률값
model.predict_proba(features_worst[250:255])

array([[0.90332419, 0.09667581],
       [0.08007073, 0.91992927],
       [0.95341881, 0.04658119],
       [0.40874722, 0.59125278],
       [0.54030015, 0.45969985]])

### 평가

In [47]:
target_predict = model.predict(features_worst)

In [48]:
from sklearn.metrics import accuracy_score

In [49]:
accuracy_score(target_worst, target_predict)

0.8488576449912126

### 서비스

In [50]:
import pickle

In [52]:
with open('../../../datasets/BreastCancerWisconsin_Logistic_quest.pkl', 'wb') as pkl_BCD :
    pickle.dump(obj=model, file=pkl_BCD)
    pass

##### .py 서비스 
- 'texture_worst','smoothness_worst','concavity_worst':25.09, 0.1416, 0.33780
    - ⇒ 25.0, 0.151, 0.35
        - 결과: Predicted diagnosis : [1]